# Scraping CoinAfrique — BeautifulSoup (Version Refactorisée)
Scraping des 4 catégories avec `requests` + `BeautifulSoup`, nettoyage des données et stockage dans une base SQLite unifiée.

In [1]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

BASE_URL = "https://sn.coinafrique.com/categorie/"

## Fonction de scraping générique

In [3]:
def scrape_page(slug, page):
    """
    Scrape une seule page d'une catégorie CoinAfrique.

    Args:
        slug (str) : identifiant de la catégorie (ex: 'vetements-homme')
        page (int) : numéro de page

    Returns:
        list[dict] : liste d'annonces avec les clés
                     categorie, nom, prix, adresse, image_lien
    """
    url = f"{BASE_URL}{slug}?page={page}"
    data = []
    try:
        response = requests.get(url, headers=HEADERS, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        containers = soup.find_all("div", class_="col s6 m4 l3")

        for container in containers:
            try:
                a_tag      = container.find("a", title=True)
                nom        = a_tag["title"].strip() if a_tag else ""

                prix_el    = container.find("p", class_="ad__card-price")
                prix       = prix_el.get_text(strip=True) if prix_el else ""

                adr_el     = container.find("p", class_="ad__card-location")
                adresse    = (
                    adr_el.get_text(separator=" ", strip=True)
                    .replace("location_on", "")
                    .strip()
                    if adr_el else ""
                )

                img_el     = container.find("img", class_="ad__card-img")
                image_lien = (
                    img_el.get("src") or img_el.get("data-src") or ""
                    if img_el else ""
                )

                data.append({
                    "categorie"  : slug,
                    "nom"        : nom,
                    "prix"       : prix,
                    "adresse"    : adresse,
                    "image_lien" : image_lien
                })
            except Exception:
                continue

    except Exception as e:
        print(f"  [!] Erreur page {page} ({slug}) : {e}")

    return data


def scrape_categorie(slug, nb_pages=9):
    """
    Scrape toutes les pages d'une catégorie et retourne un DataFrame.

    Args:
        slug     (str) : identifiant de la catégorie
        nb_pages (int) : nombre de pages à scraper

    Returns:
        pd.DataFrame
    """
    all_data = []
    for page in range(1, nb_pages + 1):
        rows = scrape_page(slug, page)
        all_data.extend(rows)
        print(f"  [{slug}] Page {page}/{nb_pages} — {len(rows)} annonces")
        time.sleep(1)
    return pd.DataFrame(all_data)

## PARTIE 1 : Exploration rapide (5 pages par catégorie)

In [4]:
CATEGORIES = [
    "vetements-homme",
    "chaussures-homme",
    "vetements-enfants",
    "chaussures-enfants",
]

df_explore = pd.DataFrame()

for slug in CATEGORIES:
    print(f"\n--- {slug} ---")
    df_cat = scrape_categorie(slug, nb_pages=5)
    df_explore = pd.concat([df_explore, df_cat], ignore_index=True)

print(f"\nTotal collecté : {len(df_explore)} annonces")
df_explore.head()


--- vetements-homme ---
  [vetements-homme] Page 1/5 — 84 annonces
  [vetements-homme] Page 2/5 — 84 annonces
  [vetements-homme] Page 3/5 — 84 annonces
  [vetements-homme] Page 4/5 — 84 annonces
  [vetements-homme] Page 5/5 — 84 annonces

--- chaussures-homme ---
  [chaussures-homme] Page 1/5 — 84 annonces
  [chaussures-homme] Page 2/5 — 84 annonces
  [chaussures-homme] Page 3/5 — 84 annonces
  [chaussures-homme] Page 4/5 — 84 annonces
  [chaussures-homme] Page 5/5 — 84 annonces

--- vetements-enfants ---
  [vetements-enfants] Page 1/5 — 84 annonces
  [vetements-enfants] Page 2/5 — 84 annonces
  [vetements-enfants] Page 3/5 — 84 annonces
  [vetements-enfants] Page 4/5 — 84 annonces
  [vetements-enfants] Page 5/5 — 84 annonces

--- chaussures-enfants ---
  [chaussures-enfants] Page 1/5 — 84 annonces
  [chaussures-enfants] Page 2/5 — 84 annonces
  [chaussures-enfants] Page 3/5 — 84 annonces
  [chaussures-enfants] Page 4/5 — 84 annonces
  [chaussures-enfants] Page 5/5 — 84 annonces

Tot

,categorie,nom,prix,adresse,image_lien
0,vetements-homme,Nuasame pour homme,20 000CFA,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5758109_u...
1,vetements-homme,Abaya homme,10 000CFA,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5779842_u...
2,vetements-homme,Abaya homme,10 000CFA,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5779835_u...
3,vetements-homme,Djalabe homme,8 500CFA,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5737158_u...
4,vetements-homme,Ensembles Jalabas Homme,9 000CFA,"Dakar, Sénégal",https://images.coinafrique.com/thumb_5776991_u...


## PARTIE 2 : Nettoyage et vérification des données

In [5]:
# Taille du dataset
print("Shape :", df_explore.shape)

# Types des variables
print("\nTypes :")
print(df_explore.dtypes)

# Valeurs manquantes
print("\nValeurs manquantes :")
print(df_explore.isna().sum())

# Doublons
print("\nDoublons :", df_explore.duplicated().sum())

# Répartition par catégorie
print("\nRépartition par catégorie :")
print(df_explore["categorie"].value_counts())

Shape : (1680, 5)

Types :
categorie     object
nom           object
prix          object
adresse       object
image_lien    object
dtype: object

Valeurs manquantes :
categorie     0
nom           0
prix          0
adresse       0
image_lien    0
dtype: int64

Doublons : 0

Répartition par catégorie :
categorie
vetements-homme       420
chaussures-homme      420
vetements-enfants     420
chaussures-enfants    420
Name: count, dtype: int64


## Pas de doublons ni de données manquantes

## PARTIE 3 : Scraping complet (9 pages) + stockage dans la base SQLite

In [6]:
# Connexion à la base unifiée
conn = sqlite3.connect("coinafrique_bs4.db")
conn.execute("""
    CREATE TABLE IF NOT EXISTS annonces (
        categorie  TEXT,
        nom        TEXT,
        prix       TEXT,
        adresse    TEXT,
        image_lien TEXT
    )
""")
conn.commit()

# Scraper et insérer chaque catégorie
for slug in CATEGORIES:
    print(f"\n--- Catégorie : {slug} ---")
    df_cat = scrape_categorie(slug, nb_pages=9)

    # Nettoyage avant insertion
    df_cat = df_cat[df_cat["nom"] != ""]
    df_cat = df_cat[df_cat["prix"] != ""]
    df_cat = df_cat.drop_duplicates()

    df_cat.to_sql("annonces", conn, if_exists="append", index=False)
    print(f"  => {len(df_cat)} annonces insérées")

print("\nScraping terminé.")


--- Catégorie : vetements-homme ---
  [vetements-homme] Page 1/9 — 84 annonces
  [vetements-homme] Page 2/9 — 84 annonces
  [vetements-homme] Page 3/9 — 84 annonces
  [vetements-homme] Page 4/9 — 84 annonces
  [vetements-homme] Page 5/9 — 84 annonces
  [vetements-homme] Page 6/9 — 84 annonces
  [vetements-homme] Page 7/9 — 84 annonces
  [vetements-homme] Page 8/9 — 84 annonces
  [vetements-homme] Page 9/9 — 84 annonces
  => 756 annonces insérées

--- Catégorie : chaussures-homme ---
  [chaussures-homme] Page 1/9 — 84 annonces
  [chaussures-homme] Page 2/9 — 84 annonces
  [chaussures-homme] Page 3/9 — 84 annonces
  [chaussures-homme] Page 4/9 — 84 annonces
  [chaussures-homme] Page 5/9 — 84 annonces
  [chaussures-homme] Page 6/9 — 84 annonces
  [chaussures-homme] Page 7/9 — 84 annonces
  [chaussures-homme] Page 8/9 — 84 annonces
  [chaussures-homme] Page 9/9 — 84 annonces
  => 756 annonces insérées

--- Catégorie : vetements-enfants ---
  [vetements-enfants] Page 1/9 — 84 annonces
  [v

## PARTIE 4 : Vérification de la base de données

In [7]:
df_all = pd.read_sql_query("SELECT * FROM annonces", conn)

print("Shape :", df_all.shape)
print("\nRépartition par catégorie :")
print(df_all["categorie"].value_counts())
print("\nValeurs manquantes :")
print(df_all.isna().sum())
print("\nDoublons :", df_all.duplicated().sum())

conn.close()
df_all.head()

Shape : (2935, 5)

Répartition par catégorie :
categorie
vetements-homme       756
chaussures-homme      756
vetements-enfants     756
chaussures-enfants    667
Name: count, dtype: int64

Valeurs manquantes :
categorie     0
nom           0
prix          0
adresse       0
image_lien    0
dtype: int64

Doublons : 0


,categorie,nom,prix,adresse,image_lien
0,vetements-homme,Nuasame pour homme,20 000CFA,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5758109_u...
1,vetements-homme,Abaya homme,10 000CFA,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5779842_u...
2,vetements-homme,Abaya homme,10 000CFA,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5779835_u...
3,vetements-homme,Djalabe homme,8 500CFA,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5737158_u...
4,vetements-homme,Ensembles Jalabas Homme,9 000CFA,"Dakar, Sénégal",https://images.coinafrique.com/thumb_5776991_u...
